# Import libraries and load dataset

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim, Bing
import requests
import re

In [2]:
# Import data
data_users_00 = pd.read_csv("data/KUL_user.csv", encoding='utf8', sep='\t')
data_users_01 = pd.read_csv("data/KUL_user_2.csv", encoding='utf8', sep='\t')
data_user = pd.concat([data_users_00, data_users_01])

In [11]:
data_users_00

,actor_id,birth_date,sex,street,number,township,zipcode
0,1860714,1973-07-08,V,Lod. van Veltemstraat,63,HERENT,3020
1,1860725,2006-03-12,V,Bovenstraat,1,LUBBEEK,3210
2,1860787,1978-12-29,M,Langeveld,152,HOLSBEEK,3220
3,1860855,2005-11-25,V,Dagobertstraat,57,LEUVEN,3000
4,1860910,1965-02-21,V,Raadhuislaan,9 /0003,LEUVEN,3010
...,...,...,...,...,...,...,...
22963,2899393,1990-01-01,M,Maria - Theresiastraat,74,LEUVEN,3000
22964,2639142,1990-01-01,M,MariaTheresiastraat,74,LEUVEN,3000
22965,2909874,1990-01-01,M,Maria - Theresiastraat,74,LEUVEN,3000
22966,2910739,1990-01-01,M,Maria - Theresiastraat,74,LEUVEN,3000


In [3]:
# Create new columns 
data_user['street_township'] = data_user['street'] + ", " + data_user['township'].str.capitalize()
data_user['township_c'] = data_user['township'].str.capitalize()

In [4]:
# Remove duplicates
data_user_clean = data_user.drop_duplicates()

# Extract the house number
data_user_clean['number_new'] = data_user_clean['number'].str.split("/").str[0]

# Concatenate de Street and house number
data_user_clean['street_number'] = data_user_clean['street'] + " " + data_user_clean['number_new']  


<ipython-input-4-bb3a6a8b1c2f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_user_clean['number_new'] = data_user_clean['number'].str.split("/").str[0]
<ipython-input-4-bb3a6a8b1c2f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_user_clean['street_number'] = data_user_clean['street'] + " " + data_user_clean['number_new']


In [5]:
# Filter by readers that live in Leuven
street_township_v2 = data_user_clean[data_user_clean['zipcode'].isin(['3000', '3001','3010','3012','3018','3050'])]
 
# Amount of readers by street and house number
readers_by_street_num = street_township_v2[['street_number', 'actor_id']]
readers_by_street_num.drop_duplicates(inplace = True)
readers_by_street_num = readers_by_street_num.groupby('street_number')['actor_id'].nunique().reset_index(name='q_reader')

# Full address
street_township_v2['address'] = street_township_v2['street'] + " " + street_township_v2['number_new'] +  ", " + street_township_v2['township_c'] 

# Select columns
street_township_v2 = street_township_v2[['street','number','street_number', 'township_c', 'zipcode']]

# Remove duplicates
street_township_v2.drop_duplicates(inplace= True)


<ipython-input-5-f7a5940eeb32>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readers_by_street_num.drop_duplicates(inplace = True)
<ipython-input-5-f7a5940eeb32>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  street_township_v2['address'] = street_township_v2['street'] + " " + street_township_v2['number_new'] +  ", " + street_township_v2['township_c']
<ipython-input-5-f7a5940eeb32>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [6]:
# Create a list
street_township_v2_list = [(x, y) for x, y  in zip(street_township_v2['street_number'], 
                              street_township_v2['township_c'])] 
 

In [7]:
# Slice list
street_township_v2_list_1 = street_township_v2_list[0:1000]
street_township_v2_list_2 = street_township_v2_list[1000:2000]
street_township_v2_list_3 = street_township_v2_list[2000:4000]
street_township_v2_list_4 = street_township_v2_list[4000:6000]
street_township_v2_list_5 = street_township_v2_list[6000:8000]
street_township_v2_list_6 = street_township_v2_list[8000:10000]
street_township_v2_list_7 = street_township_v2_list[10000:12000]
street_township_v2_list_8 = street_township_v2_list[12000:]
 

In [8]:
# Not Leuven
street_township_v2_no_l = data_user_clean[~data_user_clean['zipcode'].isin(['3000', '3001','3010','3012','3018','3050'])]
 
# Select columns 
street_township_v2_no_l = street_township_v2_no_l[[ 'street_number', 'township' ]]
street_township_v2_no_l.drop_duplicates(inplace= True)


In [9]:
# Create a list
street_township_v2_no_leu_list = [(x, y) for x, y  in zip(street_township_v2_no_l['street_number'], 
                              street_township_v2_no_l['township'])] 

In [10]:
# Slice list
street_township_v2_no_leu_list_1 = street_township_v2_no_leu_list[0:2000]
street_township_v2_no_leu_list_2 = street_township_v2_no_leu_list[2000:4000]
street_township_v2_no_leu_list_3 = street_township_v2_no_leu_list[4000:6000]
street_township_v2_no_leu_list_4 = street_township_v2_no_leu_list[6000:8000]
street_township_v2_no_leu_list_5 = street_township_v2_no_leu_list[8000:10000]
 
 

In [ ]:
geolocator = Bing(api_key='#')

results = []
# Loop over the addresses
for address in street_township_v2_no_leu_list_2:
    # Construct the full address string
    full_address = f"{address[0]}, Belgium"
    
    # Use the geolocator to get the latitude and longitude coordinates
    location = geolocator.geocode(full_address,   timeout=10)
    
    # Check whether the location was found
    if location is not None:
      # Get the zip code from the location address
      #  zip_code = location.raw.get('address', {}).get('postcode')
        

        # Get the latitude and longitude
        lat, lon = location.latitude, location.longitude
    else:
        # Set the latitude and longitude to None
        lat, lon = None, None
        
    # Append the result to the list
    results.append((address[0], lat, lon))
 

In [ ]:
# Convert the results to a Pandas DataFrame and print it
address_lat_lon = pd.DataFrame(results, columns=["Street",   "Latitude", "Longitude"])
 
 

In [ ]:
#address_lat_lon.to_csv("/content/gdrive/MyDrive/data_thesis/address_no_leuven_lat_lon_2.csv")

In [ ]:
#address_lat_lon.to_csv("/content/gdrive/MyDrive/data_thesis/address_leuven_lat_lon_8.csv")